# Read Data From CSV

In [1]:
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from numpy.linalg import norm
from quaternion import Quaternion
from scipy import stats

In [2]:
#функции калибровки
def find_offsets(mpu,counts = 85):
#     calibration_set = mpu.loc[:counts]
    calibration_set =mpu 
    n = float(len(calibration_set))
    summ = []
    offsets = []
    for i in calibration_set:
        summ.append(calibration_set[i].sum())
    for s in summ:
        offsets.append(s/n)
    return offsets

def find_headers(mpu):
    headers = []
    for i in mpu:
        headers.append(i)
    return headers

def calibration(mpu,dataset):
    
    headers = find_headers(mpu)
    print(headers)
    
    off = find_offsets(mpu)
    calibrated = dataset.copy()
    #не калибровать ускорение для фильтра маджвика!
    calibrated[headers[0]] = mpu[headers[0]] - off[0]
#     calibrated[headers[1]] = mpu[headers[1]] - off[1]
    calibrated[headers[2]] = mpu[headers[2]] - off[2]
    
    calibrated[headers[3]] = mpu[headers[3]] - off[3]
    calibrated[headers[4]] = mpu[headers[4]] - off[4]
    calibrated[headers[5]] = mpu[headers[5]] - off[5]
    
    return calibrated


In [3]:
#функции для вывода графиков
def show_plots(mpu_t,mpu_b,title_t,title_b,accel=None):
    
    headers_t = find_headers(mpu_t)
    headers_b = find_headers(mpu_b)
    
    if accel == True:
        headers_t = headers_t[:3]
        headers_b = headers_b[:3]
        title_t += ' accel'
        title_b += ' accel'
    elif accel == None:
        headers_t = headers_t[:3]
        title_t +='accel'
        headers_b = headers_b[3:]
        title_b += ' gyro'
    else:
        headers_t = headers_t[3:]
        headers_b = headers_b[3:]
        title_t += ' gyro'
        title_b += ' gyro'
        


    fig = make_subplots(rows=1, cols=2, subplot_titles=(title_t,title_b))


    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_t[headers_t[0]],name=headers_t[0]),1,1)
    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_t[headers_t[1]],name=headers_t[1]),1,1)
    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_t[headers_t[2]],name=headers_t[2]),1,1)

    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_b[headers_b[0]],name=headers_b[0]),1,2)
    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_b[headers_b[1]],name=headers_b[1]),1,2)
    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_b[headers_b[2]],name=headers_b[2]),1,2)


    fig.update_layout(legend_orientation="h",
                      legend=dict(x=.5, xanchor="center"),
                      hovermode="x",
                      margin=dict(l=0, r=0, t=40, b=0))
    fig.show()


def show_plot(mpu_t,title_t,accel=None):
    
    headers_t = find_headers(mpu_t)
    
    if accel == True:
        headers_t = headers_t[:3]
        title_t += ' accel'
    elif accel == None:
        headers_t = headers_t
    else:
        headers_t = headers_t[3:]
        title_t += ' gyro'
        


    fig = go.Figure() 
    fig.update_layout(title=title_t)
    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_t[headers_t[0]],name=headers_t[0]))
    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_t[headers_t[1]],name=headers_t[1]))
    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_t[headers_t[2]],name=headers_t[2]))

    fig.show()


In [4]:
def read_csv_file(file_path):
    df = pd.read_csv(file_path,sep = ' ',header = None)
    df.columns = ["s1","bt_ax","bt_ay","bt_az","bt_gx","bt_gy","bt_gz",
                 "s2","bb_ax","bb_ay","bb_az","bb_gx","bb_gy","bb_gz",
                 "s3","lt_ax","lt_ay","lt_az","lt_gx","lt_gy","lt_gz",
                 "s4","lb_ax","lb_ay","lb_az","lb_gx","lb_gy","lb_gz",
                "s5","rt_ax","rt_ay","rt_az","rt_gx","rt_gy","rt_gz",
                 "s6","rb_ax","rb_ay","rb_az","rb_gx","rb_gy","rb_gz",
                 ]

    return df

df = read_csv_file('dataset/__Vadim.csv')


df

,s1,bt_ax,bt_ay,bt_az,bt_gx,bt_gy,bt_gz,s2,bb_ax,bb_ay,...,rt_gx,rt_gy,rt_gz,s6,rb_ax,rb_ay,rb_az,rb_gx,rb_gy,rb_gz
0,s1,0.308852,-9.696517,-2.147599,-0.014389,0.001865,-0.018919,s2,0.074220,10.29986,...,0.018119,0.041302,0.001066,s6,-2.731784,9.749189,-0.694318,0.000666,-0.027712,-0.007461
1,s1,0.284910,-9.744401,-2.020706,-0.027046,0.000666,-0.020118,s2,0.093374,10.29028,...,0.018786,0.050761,-0.002531,s6,-2.703054,9.756371,-0.514753,-0.000266,-0.009859,-0.010925
2,s1,0.313640,-9.725247,-2.190695,-0.030643,-0.001732,-0.015855,s2,0.031125,10.31662,...,0.018652,0.043966,0.002531,s6,-2.810793,9.761160,-0.658405,-0.001732,-0.016121,-0.007061
3,s1,0.260968,-9.727641,-2.114080,-0.002665,0.015322,-0.012257,s2,0.064643,10.32380,...,0.014389,0.040502,0.007328,s6,-2.758120,9.737218,-0.608127,0.003064,-0.038371,-0.006262
4,s1,0.373495,-9.722853,-2.106898,-0.018519,0.019851,-0.009726,s2,0.028730,10.28549,...,0.018386,0.032508,0.009593,s6,-2.770091,9.753978,-0.577003,-0.008394,-0.035440,-0.002265
5,s1,0.308852,-9.706094,-2.152387,-0.022516,0.009326,-0.009992,s2,0.038307,10.27352,...,0.018119,0.042900,0.008394,s6,-2.801216,9.718064,-0.538695,0.003064,-0.035440,0.000933
6,s1,0.313640,-9.682152,-2.200271,-0.036772,0.013323,-0.009726,s2,0.079009,10.31901,...,0.017853,0.040236,0.004263,s6,-2.767697,9.689334,-0.593762,0.000000,-0.033041,-0.006795
7,s1,0.313640,-9.670180,-2.128445,-0.033974,0.007061,-0.015588,s2,0.079009,10.29267,...,0.017054,0.023982,0.006262,s6,-2.705448,9.782708,-0.644040,-0.001199,-0.037838,-0.005462
8,s1,0.366313,-9.694122,-2.260126,-0.031309,0.029577,-0.007061,s2,0.023942,10.27112,...,0.020518,0.035839,0.009326,s6,-2.748544,9.775525,-0.543484,0.001599,-0.041168,-0.004530
9,s1,0.258574,-9.713276,-2.070984,-0.024381,0.012391,-0.011591,s2,0.069432,10.27352,...,0.013190,0.047830,0.006262,s6,-2.703054,9.763555,-0.591368,0.003464,-0.028645,-0.006129


In [5]:
#each  IMU for calibration
def parse_to_sensors(df):
    bt = df[["bt_ax","bt_ay","bt_az","bt_gx","bt_gy","bt_gz"]]
    bb = df[["bb_ax","bb_ay","bb_az","bb_gx","bb_gy","bb_gz"]]
    lt = df[["lt_ax","lt_ay","lt_az","lt_gx","lt_gy","lt_gz"]]
    lb = df[["lb_ax","lb_ay","lb_az","lb_gx","lb_gy","lb_gz"]]
    rt = df[["rt_ax","rt_ay","rt_az","rt_gx","rt_gy","rt_gz"]]
    rb = df[["rb_ax","rb_ay","rb_az","rb_gx","rb_gy","rb_gz"]]
    
#     show_plot(lb,'Non calibrated left-bot ',True)
#     show_plot(lb,'Non calibrated left-bot ',False)
    return bt,bb,lt,lb,rt,rb


# Filter outliers

In [6]:
def filter_outliers(df_local):
    
    outl = find_headers(df_local)
    df= df_local.copy()
    for o in outl:
        median = df[o].median()
        std = df[o].std()
        df.loc[(df[o] - median).abs() > std,o]=np.nan
        df[o].fillna(median, inplace=True)
    return df

def find_outliers(df):
    
    bt,bb,lt,lb,rt,rb = parse_to_sensors(df)
    
    rb_outl = filter_outliers(rb)
    rt_outl =filter_outliers(rt)
    lb_outl = filter_outliers(lb)
    lt_outl = filter_outliers(lt)
    bb_outl = filter_outliers(bb)
    bt_outl = filter_outliers(bt)
    

#     show_plot(lb,'With outliers ',False)
#     show_plot(lb_outl,'With outliers ',False)
#     show_plot(calibrated_lb,'Without outliers',False)

    return rb_outl,rt_outl,lb_outl,lt_outl,bb_outl,bt_outl

# def all_calibrate(df):
#     calibration()

# calibration_frame = find_outliers(df)
# calibration_frame
# show_plot(lb,'With outliers ',True)
# show_plot(lb_outl,'Without outliers',True)

In [7]:
# df

 Calibrate DF of each sensor

# Calibration data

In [16]:
# df_1 = pd.read_csv('dataset/7_Danya5.csv',sep = ' ',header = None)
# df_1.columns = ["s1","bt_ax","bt_ay","bt_az","bt_gx","bt_gy","bt_gz",
#              "s2","bb_ax","bb_ay","bb_az","bb_gx","bb_gy","bb_gz",
#              "s3","lt_ax","lt_ay","lt_az","lt_gx","lt_gy","lt_gz",
#              "s4","lb_ax","lb_ay","lb_az","lb_gx","lb_gy","lb_gz",
#             "s5","rt_ax","rt_ay","rt_az","rt_gx","rt_gy","rt_gz",
#              "s6","rb_ax","rb_ay","rb_az","rb_gx","rb_gy","rb_gz",
#              ]
# df


df_1 = read_csv_file('dataset/__Vadim1.csv')

def all_calibrate(calibration_set,dataset):
    
    bt_1,bb_1,lt_1,lb_1,rt_1,rb_1=parse_to_sensors(dataset) 
    
    rb_outl,rt_outl,lb_outl,lt_outl,bb_outl,bt_outl =  find_outliers(calibration_set)
    
    calibrated_rb = calibration(rb_outl,rb_1)
    calibrated_rt = calibration(rt_outl,rt_1)
    calibrated_lb = calibration(lb_outl,lb_1)
    calibrated_lt = calibration(lt_outl,lt_1)
    calibrated_bb = calibration(bb_outl,bb_1)
    calibrated_bt = calibration(bt_outl,bt_1)
#    dataframe = pd.concat([calibrated_bt,calibrated_bb,calibrated_lt,calibrated_lb,calibrated_rt,calibrated_rb],axis = 1)
    show_plot(lb_1,'Non calibrtated ',False)
    show_plot(calibrated_lb,'calibrated ',False)
    return calibrated_lb
    


calibrated_lb = all_calibrate(df,df_1)

# def all_calibrated(df):
  
# #первый фрейм калибровочный файл
# calibrated_lb = calibration(lt_outl,lt_1)
# #calibrated_lb = lb_1



['rb_ax', 'rb_ay', 'rb_az', 'rb_gx', 'rb_gy', 'rb_gz']
['rt_ax', 'rt_ay', 'rt_az', 'rt_gx', 'rt_gy', 'rt_gz']
['lb_ax', 'lb_ay', 'lb_az', 'lb_gx', 'lb_gy', 'lb_gz']
['lt_ax', 'lt_ay', 'lt_az', 'lt_gx', 'lt_gy', 'lt_gz']
['bb_ax', 'bb_ay', 'bb_az', 'bb_gx', 'bb_gy', 'bb_gz']
['bt_ax', 'bt_ay', 'bt_az', 'bt_gx', 'bt_gy', 'bt_gz']


# find orientation

In [9]:
import madgwickahrs as mg
from quaternion import Quaternion

In [18]:
#find only with gyro 

pitch = np.zeros(len(calibrated_lb))
yaw = np.zeros(len(calibrated_lb))
roll = np.zeros(len(calibrated_lb))

for i in range(len(calibrated_lb)-1):
    delta_t = 0.06
    
    pitch[i+1] = pitch[i] + calibrated_lb['lb_gy'][i]*delta_t
    yaw[i+1] = yaw[i]+calibrated_lb['lb_gz'][i]*delta_t
    roll[i+1] = roll[i]+calibrated_lb['lb_gx'][i]*delta_t
    

orientation_d = pd.DataFrame({'roll':np.rad2deg(roll),'pitch':np.rad2deg(pitch),'yaw':np.rad2deg(yaw)})

show_plot(orientation_d,'Orientation only with  gyro')


In [11]:
# Madgwick filter

In [12]:
def find_orientation(calibrated_mpu):
    
    headers = find_headers(calibrated_mpu)
        
    quaternions = []

    heading = mg.MadgwickAHRS(sampleperiod = 0.06,beta = 1)
    for i,row in calibrated_mpu.iterrows():
        gyro_d  = []
        accel_d = []
        gyro_d.append(row[headers[3]])
        gyro_d.append(row[headers[4]])
        gyro_d.append(row[headers[5]])
        accel_d.append(row[headers[0]])
        accel_d.append(row[headers[1]])
        accel_d.append(row[headers[2]])

        heading.update_imu(gyro_d,accel_d)
        quaternions.append(heading.quaternion._get_q())

    # print(quaternions)
    pitch = []
    roll = []
    yaw = []
    for i in range(len(quaternions)):
        pitch.append(np.arcsin(2 * quaternions[i][1] * quaternions[i][2] + 2 * quaternions[i][0] * quaternions[i][3]))
        if np.abs(quaternions[i][1] * quaternions[i][2] + quaternions[i][3] * quaternions[i][0] - 0.5) < 1e-8:
            roll.append(0) 
            yaw.append(2 * np.arctan2(quaternions[i][1], quaternions[i][0]))
        elif np.abs(quaternions[i][1] * quaternions[i][2] + quaternions[i][3] * quaternions[i][0] + 0.5) < 1e-8:
            roll.append(-2 * np.arctan2(quaternions[i][1], quaternions[i][0]))
            yaw.append(0)
        else:
            roll.append(np.arctan2(2 * quaternions[i][0] * quaternions[i][1] - 2 * quaternions[i][2] *quaternions[i][3], 1 - 2 * quaternions[i][1] ** 2 - 2 *quaternions[i][3] ** 2))
            yaw.append(np.arctan2(2 * quaternions[i][0] * quaternions[i][2] - 2 * quaternions[i][1] * quaternions[i][3], 1 - 2 * quaternions[i][2] ** 2 - 2 * quaternions[i][3] ** 2))


    orientation_d = pd.DataFrame({'roll':np.rad2deg(roll),'pitch':np.rad2deg(pitch),'yaw':np.rad2deg(yaw)})
    return orientation_d

In [17]:
orientation_lb = find_orientation(calibrated_lb)
show_plot(orientation_lb,'Orientation left_bot sensor')

In [14]:


# quaternions = []

# heading = mg.MadgwickAHRS(sampleperiod = 0.06,beta = 1)
# for i,row in calibrated_lb.iterrows():
#     gyro_d  = []
#     accel_d = []
#     gyro_d.append(row['lb_gx'])
#     gyro_d.append(row['lb_gy'])
#     gyro_d.append(row['lb_gz'])
#     accel_d.append(row['lb_ax'])
#     accel_d.append(row['lb_ay'])
#     accel_d.append(row['lb_az'])

#     heading.update_imu(gyro_d,accel_d)
#     quaternions.append(heading.quaternion._get_q())

# # print(quaternions)
# pitch = []
# roll = []
# yaw = []
# for i in range(len(quaternions)):
#     pitch.append(np.arcsin(2 * quaternions[i][1] * quaternions[i][2] + 2 * quaternions[i][0] * quaternions[i][3]))
#     if np.abs(quaternions[i][1] * quaternions[i][2] + quaternions[i][3] * quaternions[i][0] - 0.5) < 1e-8:
#         roll.append(0) 
#         yaw.append(2 * np.arctan2(quaternions[i][1], quaternions[i][0]))
#         print('thth')
#     elif np.abs(quaternions[i][1] * quaternions[i][2] + quaternions[i][3] * quaternions[i][0] + 0.5) < 1e-8:
#         roll.append(-2 * np.arctan2(quaternions[i][1], quaternions[i][0]))
#         yaw.append(0)
#     else:
#         roll.append(np.arctan2(2 * quaternions[i][0] * quaternions[i][1] - 2 * quaternions[i][2] *quaternions[i][3], 1 - 2 * quaternions[i][1] ** 2 - 2 *quaternions[i][3] ** 2))
#         yaw.append(np.arctan2(2 * quaternions[i][0] * quaternions[i][2] - 2 * quaternions[i][1] * quaternions[i][3], 1 - 2 * quaternions[i][2] ** 2 - 2 * quaternions[i][3] ** 2))


# orientation_d = pd.DataFrame({'roll':roll,'pitch':pitch,'yaw':yaw})

In [ ]:
# orientation_d

In [ ]:
# show_plot(lb,'gyro_lb',False)

In [ ]:
# fig = go.Figure() 
# fig.update_layout(title='roll lb')

# fig.add_trace(go.Scatter(x=orientation_d.index, y=np.rad2deg(orientation_d['roll']),name='roll'))
# fig.add_trace(go.Scatter(x=orientation_d.index, y=np.rad2deg(orientation_d['pitch']),name='pitch'))
# fig.add_trace(go.Scatter(x=orientation_d.index, y=np.rad2deg(orientation_d['yaw']),name='yaw'))
# # fig.add_trace(go.Scatter(x=agles.index, y=agles['x'],name='gyro'))

# fig.show()

In [ ]:
#(-0.02213089742574257, 0.01005698124752475, -0.013309934386138617)